### Подготовка данных для LDA

Выберем нужные нам темы и необходимое число статей для каждой темы. Словарь dataOption в качестве ключей хранит название темы, а в качестве значений число статей для соответствующей темы.

In [103]:
class optionsLDA(object):
    def __init__(self, path, topics, nWordPerTopic=100):
        self.alpha = 0.1
        self.beta = 0.1
        
        self.topics = topics
        # сформировать словари wordid, docid, docs
        self.idToDoc = {}
        self.wordToId = {}
        self.docs = []
        last_wordid = -1
        last_docid = -1
        
        
        self.nArticles = []
        for topicName in topics:
            list_articles = os.listdir(path + topicName)
            list_articles = np.random.permutation(list_articles)
            
            nWords = 0
            nArticles = 0
            
            for article in list_articles: 
                # Создадим id для статьи
                last_docid += 1
                self.idToDoc[last_docid] = topicName + '/' + article

                nArticles += 1
                # Обработка статьи
                doc = []
                with codecs.open(path + topicName + '/' + article, 'r', 'utf8') as inf:
                    for line in inf:
                        line = line.split()
                        nWords += len(line)
                        for w in line:
                            if w not in self.wordToId.keys():
                                last_wordid += 1
                                self.wordToId[w] = last_wordid
                            doc.append(self.wordToId[w])
                self.docs.append(doc)
                if nWords > nWordPerTopic:
                    break  
            self.nArticles.append(nArticles)
        self.M = len(self.docs)
        self.V = len(self.wordToId.keys())
    def info(self):
        print "Количество статей в каждой категории: "
        for t,a in zip(self.topics, self.nArticles):
            print u'[%s:%d] '%(t,a),
            
        print "\nРазмер словаря: %d"%(self.V)
        print "Количество статей: %d"%(self.M)
        print "\nТОР 20 самых часто встречающихся слов из каждой темы:"

In [119]:
path = "./preprocessedNews/"
for d in os.listdir(path):
    print "[%s]"%d, 

[Власть] [Технологии] [Происшествия] [Строительство] [Деньги] [Работа] [Недвижимость] [Туризм] [ЖКХ] [Общество] [Авто] [Финансы] [Спорт] [Город] [Бизнес] [Доброе дело]


In [129]:
options = optionsLDA(path, [u'Авто', u'Спорт', u'Доброе дело'], 10000)

In [130]:
options.info()

Количество статей в каждой категории: 
[Авто:96]  [Спорт:114]  [Доброе дело:55]  
Размер словаря: 7683
Количество статей: 265

ТОР 20 самых часто встречающихся слов из каждой темы:


# LDA
## Реализация класса LDA


 <img src=smoothed_lda.png>

In [124]:
class LDA(object):
    def __init__(self, K, option):
        self.K = K
        self.alpha = option.alpha
        self.beta = option.beta
        
        self.Voc = {}
        for k,v in option.wordToId.items():
            self.Voc[v] = k
        self.V = option.V
        
        
        self.docs = options.docs
        self.M = option.M
    
        
        self.z_mn = []                                 # тема n-го слова в документе m
        self.n_mz = np.zeros((self.M, self.K)) + self.alpha # число слов в документе m с темой z
        self.n_zt = np.zeros((self.K, self.V)) + self.beta  # сколько раз слово t встречается в теме z
        self.n_z  = np.zeros((self.K)) + self.V*self.beta   # число слов в каждой теме
        
        # смотрим все документы
        for m, doc in enumerate(self.docs):
            # для каждого слова выбираем тему
            z_n = []                           # z_n[i] хранит тему для i-го слова
            
            # смотрим все слова в этом документе, wordid - ид слова, nw - сколько раз слово встречается в документе
            for t in doc:                
                # выбираем тему
                #z = np.random.randint(0, self.K)
                p_z = self.n_zt[:,t] * self.n_mz[m] / self.n_z
                z = np.random.multinomial(1, p_z / p_z.sum()).argmax()

                z_n.append(z)

                # обновляем матрицы
                self.n_mz[m, z]      += 1.0
                self.n_zt[z, t]      += 1.0
                self.n_z[z]          += 1.0
            self.z_mn.append(np.array(z_n))
            
    def iteration(self):
        """Одна итерация"""
        # смотрим все документы
        for m,doc in enumerate(self.docs):
            for n, t in enumerate(doc):
                # выбираем тему
                z = self.z_mn[m][n]
                #print self.n_mz[docid, z], self.n_zt[z, wordid], self.n_z[z]
                self.n_mz[m,z]   -= 1.0
                self.n_zt[z, t] -= 1.0
                self.n_z[z]          -= 1.0
                #print self.n_mz[docid, z], self.n_zt[z, wordid], self.n_z[z]

                #выбриаем новую тему
                p_z = (self.n_zt[:,m]) * (self.n_mz[m]) / (self.n_z)

                #print p_z
                #print p_z.sum()
                new_z = np.random.multinomial(1, p_z / p_z.sum()).argmax()

                self.z_mn[m][n] = new_z
                self.n_mz[m, new_z]      += 1.0
                self.n_zt[new_z, t]      += 1.0
                self.n_z[new_z]          += 1.0

    def get_phi(self):
        return self.n_zt / self.n_z[:, np.newaxis]
    
    def perplexity(self):
        phi = self.get_phi()
        N = 0
        logPerplexity = 0
        
        for m,doc in enumerate(self.docs):
            theta = self.n_mz[m] /(len(doc) + self.K * self.alpha)
            for w in doc:
                logPerplexity -= np.log(np.inner(phi[:,w], theta))
            N += len(doc)
        return np.exp(logPerplexity / N)
    
    
    def learn(self, n_iter):
        last_perplexity = self.perplexity()
        print "Начальное значение perplexity: %f" % last_perplexity
        for i in range(n_iter):
            self.iteration()
            perplexity = self.perplexity()
            print "Итерация %d: perplexity=%f"%(i+1,perplexity)
            #if last_perplexity < perplexity:
            #    print "model is learned"
            #    return
            if perplexity - last_perplexity > 20:
                print "OK"
                return
            if last_perplexity > perplexity:
                perplexity = last_perplexity
        
    def get_wt_distribution(self):
        phi = self.get_phi()
        for i in range(self.K):
            print "\n\ntopic %d"%i
            for w in np.argsort(-phi[i])[:20]:
                print "+ %f*%s"%(phi[i,w], self.Voc[w]),
        
        
        

In [131]:
mylda = LDA(3, options)

In [134]:
mylda.learn(10)

Начальное значение perplexity: 2774.938378
Итерация 1: perplexity=2780.938070
Итерация 2: perplexity=2747.995510
Итерация 3: perplexity=2727.967620
Итерация 4: perplexity=2737.467532
Итерация 5: perplexity=2732.245950
Итерация 6: perplexity=2712.264593
Итерация 7: perplexity=2707.046242
Итерация 8: perplexity=2695.743503
Итерация 9: perplexity=2674.410200
Итерация 10: perplexity=2663.553543


In [135]:
for t in options.topics:
    print "[%s]"%t,
mylda.get_wt_distribution()

[Авто] [Спорт] [Доброе дело] 

topic 0
+ 0.006179*россия + 0.005732*команда + 0.004749*матч + 0.004033*чемпионат + 0.003944*ребёнок + 0.003854*стать + 0.003765*зенит + 0.003675*петербург + 0.003497*клуб + 0.003139*автомобиль + 0.002960*фонд + 0.002871*игра + 0.002692*рубль + 0.002334*участие + 0.002334*ска + 0.002334*напомнить + 0.002245*сборный + 0.002155*российский + 0.002155*проект + 0.002155*тренер 

topic 1
+ 0.005376*ребёнок + 0.005150*петербург + 0.004698*рубль + 0.004189*фонд + 0.004019*россия + 0.003963*дом + 0.003567*благотворительный + 0.003454*район + 0.003115*проект + 0.002719*место + 0.002663*стать + 0.002380*организация + 0.002380*сообщать + 0.002323*автомобиль + 0.002267*улица + 0.002154*социальный + 0.002154*зенит + 0.002097*матч + 0.002097*система + 0.002041*чемпионат 

topic 2
+ 0.005374*россия + 0.005374*петербург + 0.004811*автомобиль + 0.004248*транспортный + 0.003686*рубль + 0.003686*матч + 0.003404*чемпионат + 0.003404*место + 0.002560*средство + 0.002560*органи

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True